In [1]:
from __future__ import print_function

from rdkit import Chem
from rdkit.Chem import AllChem

In [9]:
## Alcohols

KETONE_2 = AllChem.ReactionFromSmarts('[CH:1][O:2]>>[C:1]=[O:2]')
CARBOXY_1 = AllChem.ReactionFromSmarts('[CH2:1]O>>[C:1](=O)O')
ALDEHYDE_1 = AllChem.ReactionFromSmarts('[CH2:1][O:2]>>[C:1](=[O:2])')

CHLORO_1_2 = AllChem.ReactionFromSmarts('[CH,CH2:1]O>>[C:1]Cl')
CHLORO_3 = AllChem.ReactionFromSmarts('[CH0:1]O>>[C:1]Cl')
BROMO_1_2 = AllChem.ReactionFromSmarts('[CH,CH2:1]O>>[C:1]Br')
BROMO_3 = AllChem.ReactionFromSmarts('[CH0:1]O>>[C:1]Br')
IODO_3 = AllChem.ReactionFromSmarts('[CH0:1]O>>[C:1]I')

TOSYL = AllChem.ReactionFromSmarts('[C!H0:1][O:2].[Cl:3][S:4]>>[O:2][S:4]')

ETHER = AllChem.ReactionFromSmarts('[C!H0:1][O:2].[Cl,Br,I:3][CH2,CH3:4]>>[O:2][C:4]')


alcohol_reactions_list = [
    
    ('KETONE_2', KETONE_2, [''], ['[O-][Cr](=O)(=O)O[Cr](=O)(=O)[O-].[Na+].[Na+].OS(O)(=O)=O', \
                                  '[O-][Cr](=O)(=O)O[Cr](=O)(=O)[O-].[K+].[K+].OS(O)(=O)=O']),
    ('CARBOXY_1', CARBOXY_1, [''], ['[O-][Cr](=O)(=O)O[Cr](=O)(=O)[O-].[Na+].[Na+].OS(O)(=O)=O', \
                                    '[O-][Cr](=O)(=O)O[Cr](=O)(=O)[O-].[K+].[K+].OS(O)(=O)=O']),
    ('ALDEHYDE_1', ALDEHYDE_1, [''], ['Cl.O=[Cr](=O)=O.c1ccncc1']),
    
    ('CHLORO_1_2', CHLORO_1_2, [''], ['O=S(Cl)Cl.c1ccncc1', 'ClP(Cl)Cl', 'ClP(Cl)(Cl)(Cl)Cl']),
    ('CHLORO_3', CHLORO_3, [''], ['Cl']),
    ('BROMO_1_2', BROMO_1_2, [''], ['BrP(Br)Br']),
    ('BROMO_3', BROMO_3, [''], ['Br']),
    ('IODO_3', IODO_3, [''], ['I']),
    
    ('TOSYL', TOSYL, ['Cc1ccc(cc1)S(Cl)(=O)=O'], ['c1ccncc1']),
    
    ('ETHER', ETHER, [''], ['[Na]', '[K]', '[H-].[Na+]']),
    
]

In [12]:
import copy

def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def block(ch, smiles):
    return (ch + cano(smiles)) if (smiles != '') else ''

# choose alcohol(tmp)
alcohol_smi = 'CC(C)O'
alcohol = Chem.MolFromSmiles(alcohol_smi)

# choose alkyl halide (tmp)
RX_smi = 'CCCBr'

# general reactions
for reaction in alcohol_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'ETHER':
            reagent = RX_smi
        if reagent == '': products = reaction[1].RunReactants((alcohol,))
        else: products = reaction[1].RunReactants((alcohol, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print(cano(alcohol_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

KETONE_2
CC(C)O>O=S(=O)(O)O.O=[Cr](=O)([O-])O[Cr](=O)(=O)[O-].[Na+].[Na+]>CC(C)=O
CC(C)O>O=S(=O)(O)O.O=[Cr](=O)([O-])O[Cr](=O)(=O)[O-].[K+].[K+]>CC(C)=O
CARBOXY_1
ALDEHYDE_1
CHLORO_1_2
CC(C)O>O=S(Cl)Cl.c1ccncc1>CC(C)Cl
CC(C)O>ClP(Cl)Cl>CC(C)Cl
CC(C)O>ClP(Cl)(Cl)(Cl)Cl>CC(C)Cl
CHLORO_3
BROMO_1_2
CC(C)O>BrP(Br)Br>CC(C)Br
BROMO_3
IODO_3
TOSYL
CC(C)O.Cc1ccc(S(=O)(=O)Cl)cc1>c1ccncc1>Cc1ccc(cc1)S(=O)(=O)O
ETHER
CC(C)O.CCCBr>[Na]>CCCO
CC(C)O.CCCBr>[K]>CCCO
CC(C)O.CCCBr>[H-].[Na+]>CCCO
